In [1]:
from __future__ import print_function, division
import os
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from reData import *
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch.nn.functional as F
import torch.nn as nn
import time
from tqdm import tqdm
import torch.utils.data as utils
import torchvision.models as models
from torch.optim import lr_scheduler

## 加载特征数据 / 定义全连接网络融合特征

- Net为融合特征的简单的一层全连接
- 把特征数据从四维转成两维来输出到Net

In [2]:
data_dir = 'data/dog_breed'        
label_file, train_dir, test_dir = 'labels.csv', 'train', 'test'
input_dir, batch_size, valid_ratio = 'train_valid_test', 128, 0.1
class Net(nn.Module):
    def __init__(self,**kwargs):
        super(Net,self).__init__(**kwargs)
        self.net = nn.Sequential(
            nn.Linear(in_features=4768, out_features=120, bias=True),
#             nn.ReLU(True),
#             nn.Dropout(0.5),
#             nn.Linear(in_features=300, out_features=120, bias=True)
        )
    def forward(self, x):
        return self.net(x)
    
EPOCH = 200
batch_size = 64

train_pt = torch.load('train_r152i3.pt')
valid_pt = torch.load('valid_r152i3.pt')
input_pt = torch.load('input_r152i3.pt')
test_pt = torch.load('test_r152i3.pt')

#reshape [b,c,1,1] to [b,c]
for i in range(2):
    train_pt[0] = train_pt[0].view(train_pt[0].shape[0],train_pt[0].shape[1])
    valid_pt[0] = valid_pt[0].view(valid_pt[0].shape[0],valid_pt[0].shape[1])
    input_pt[0] = input_pt[0].view(input_pt[0].shape[0],input_pt[0].shape[1])
    test_pt[0] = test_pt[0].view(test_pt[0].shape[0],test_pt[0].shape[1])
    

train_dl = DataLoader(utils.TensorDataset(train_pt[0],train_pt[1]), batch_size=batch_size,shuffle=True)
valid_dl = DataLoader(utils.TensorDataset(valid_pt[0],valid_pt[1]), batch_size=batch_size,shuffle=True)
train_valid_dl = DataLoader(utils.TensorDataset(input_pt[0],input_pt[1]), batch_size=batch_size,shuffle=True)
test_dl = DataLoader(utils.TensorDataset(test_pt[0],test_pt[1]), batch_size=batch_size,shuffle=False)

## 训练函数

In [3]:
def train(net,EPOCH,loss_func,optimizer,train_valid_dl,train_dl,valid_dl,use_valid = True):
    def evaluate_lossAndAcc(data_iter, net):
        l_sum,acc = 0.0, 0.0
        for X, y in data_iter:
            #cuda
            X = X.cuda()
            y = y.cuda()
            y_pre = net(X)
            #calculate acc
            pred_y = torch.max(y_pre, 1)[1].cpu().data.numpy()
            accuracy = float((pred_y == y.data.cpu().numpy()).astype(int).sum()) / float(y.size(0))
            acc += accuracy
            #calculate sum
            l_sum += loss_func(y_pre, y).data.cpu().numpy()
        return l_sum / len(data_iter), acc / len(data_iter)

    if use_valid:
        data_iter = train_dl
    else:
        data_iter = train_valid_dl
        
    for epoch in range(1,EPOCH+1):
        start = time.time()
        train_lsum, n= 0.0, 0
        for step,(X,y) in enumerate(data_iter):
            X = X.cuda()
            y = y.cuda()
            output = net(X)
            loss = loss_func(output,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_lsum += loss.cpu().data.numpy()
            n += float(y.size(0))
        if epoch % 5 == 0:
            train_loss,train_acc = evaluate_lossAndAcc(train_dl, net)
            if use_valid:
                valid_loss,valid_acc = evaluate_lossAndAcc(valid_dl, net)
                print("epoch %d | train loss : %.4f | valid loss : %.4f | train_acc : %.4f | valid_acc : %.4f | time :%.4f sec" 
                           % (epoch, train_loss, valid_loss,train_acc,valid_acc,(time.time() - start)))
            else:
                 print("epoch %d | train loss : %.4f | train_acc : %.4f | time :%.4f sec" 
                           % (epoch, train_loss,train_acc,(time.time() - start)))
#         else:
#             print("epoch %d | train loss : %.4f | time :%.4f sec" 
#                        % (epoch, train_lsum / n,(time.time() - start)))

In [ ]:
## 提取各种深度网络的feature层

- 提取VGG19,ResNet50,ResNet152,Desnet162的features层的输出
- 通过average pooling来调整输出为(batchsize , channel, 1 , 1)
- 通过ConcatNet 来合并所有输出

*这里只选择了三种网络vgg19 / resnet152 / desnet162 进行合并*

In [4]:
loss_func = nn.CrossEntropyLoss()
net = Net().cuda()
optimizer = torch.optim.SGD(params=net.parameters(), lr=1e-3,momentum = 0.7,weight_decay=1e-4)
#optimizer = torch.optim.Adam(params=net.parameters(),lr=1e-3,weight_decay=1e-3)
#exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
train(net,EPOCH,loss_func,optimizer,train_valid_dl,train_dl,valid_dl,use_valid = True)

epoch 5 | train loss : 0.8753 | valid loss : 0.9557 | train_acc : 0.9168 | valid_acc : 0.8815 | time :0.4157 sec
epoch 10 | train loss : 0.4923 | valid loss : 0.5814 | train_acc : 0.9380 | valid_acc : 0.8932 | time :1.5548 sec
epoch 15 | train loss : 0.3722 | valid loss : 0.4645 | train_acc : 0.9461 | valid_acc : 0.8971 | time :0.3535 sec
epoch 20 | train loss : 0.3102 | valid loss : 0.4431 | train_acc : 0.9511 | valid_acc : 0.8867 | time :0.8151 sec
epoch 25 | train loss : 0.2710 | valid loss : 0.4020 | train_acc : 0.9560 | valid_acc : 0.8880 | time :0.6144 sec
epoch 30 | train loss : 0.2428 | valid loss : 0.3784 | train_acc : 0.9603 | valid_acc : 0.8867 | time :1.5378 sec
epoch 35 | train loss : 0.2213 | valid loss : 0.3533 | train_acc : 0.9641 | valid_acc : 0.9010 | time :0.3081 sec
epoch 40 | train loss : 0.2033 | valid loss : 0.3389 | train_acc : 0.9670 | valid_acc : 0.9010 | time :0.3081 sec
epoch 45 | train loss : 0.1884 | valid loss : 0.3434 | train_acc : 0.9698 | valid_acc : 0

In [5]:
#all data (train + valid)
net = Net().cuda()
loss_func = nn.CrossEntropyLoss()
#optimizer = torch.optim.SGD(params=net.parameters(), lr=1e-4,weight_decay=1e-5)
optimizer = torch.optim.SGD(params=net.parameters(), lr=1e-3,momentum = 0.9,weight_decay=1e-4)
#optimizer = torch.optim.Adam(params=net.parameters(),lr=1e-4,weight_decay=1e-5)
train(net,EPOCH,loss_func,optimizer,train_valid_dl,train_dl,valid_dl,use_valid = False)

epoch 5 | train loss : 0.3580 | train_acc : 0.9426 | time :0.9197 sec
epoch 10 | train loss : 0.2380 | train_acc : 0.9579 | time :0.7564 sec
epoch 15 | train loss : 0.1859 | train_acc : 0.9673 | time :1.1891 sec
epoch 20 | train loss : 0.1545 | train_acc : 0.9750 | time :0.3070 sec
epoch 25 | train loss : 0.1321 | train_acc : 0.9796 | time :0.3037 sec
epoch 30 | train loss : 0.1163 | train_acc : 0.9828 | time :0.3148 sec
epoch 35 | train loss : 0.1025 | train_acc : 0.9858 | time :0.3173 sec
epoch 40 | train loss : 0.0922 | train_acc : 0.9889 | time :0.3167 sec
epoch 45 | train loss : 0.0835 | train_acc : 0.9902 | time :0.3159 sec
epoch 50 | train loss : 0.0761 | train_acc : 0.9931 | time :0.3161 sec
epoch 55 | train loss : 0.0701 | train_acc : 0.9943 | time :0.3284 sec
epoch 60 | train loss : 0.0645 | train_acc : 0.9956 | time :0.3026 sec
epoch 65 | train loss : 0.0602 | train_acc : 0.9965 | time :0.3027 sec
epoch 70 | train loss : 0.0560 | train_acc : 0.9966 | time :0.3348 sec
epoch 7

In [6]:
net = net.eval()
def prediction(data_iter,net):
    preds = []
    for X, y in data_iter:
        X = X.cuda()
        output = net(X)
        softmax = nn.Softmax(1)# softmax on each row
        pred = softmax(output).data.cpu().numpy()
        preds.extend(pred)
    return preds
preds = np.array(prediction(test_dl,net))

In [7]:
df_pred = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))
for i, c in enumerate(df_pred.columns[1:]):
    df_pred[c] = preds[:,i]
df_pred.to_csv('pred.csv', index=None)